IMA 205 Challenge Kaggle: Ilyass MEJDOUB (M6Enjoyer)
---

In [1]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)


2024-05-05 14:41:10.326397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-05 14:41:10.326443: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-05 14:41:10.449979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-05 14:41:10.614993: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-05 14:41:12.935673: W tensorflow/compiler/tf2

3 Physical GPUs, 1 Logical GPU


2024-05-05 14:41:16.802831: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 29557 MB memory:  -> device: 1, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:81:00.0, compute capability: 7.0


Preprocessing
---

In [3]:
from keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np


def append_ext(fn):
    return fn+".jpg"
    
traindf=pd.read_csv("metadataTrain.csv",dtype=str)    
traindf["ID"]=traindf["ID"].apply(append_ext)
labels = pd.DataFrame(traindf[['ID', 'CLASS']])
labels = labels.sort_values(by='ID').reset_index(drop=True)

datagen=ImageDataGenerator(zoom_range=0.1,
                           rotation_range=350,
                           validation_split=0.15)


<class 'str'>
                     ID CLASS
0      ISIC_0000000.jpg     2
1      ISIC_0000001.jpg     2
2      ISIC_0000002.jpg     1
3      ISIC_0000003.jpg     2
4      ISIC_0000004.jpg     1
...                 ...   ...
18993  ISIC_0073241.jpg     1
18994  ISIC_0073244.jpg     2
18995  ISIC_0073245.jpg     2
18996  ISIC_0073248.jpg     5
18997  ISIC_0073251.jpg     2

[18998 rows x 2 columns]


In [4]:
train_generator=datagen.flow_from_dataframe(
    dataframe=labels,
    directory="Train_all/Train",
    x_col="ID",
    y_col="CLASS",
    subset = 'training',
    batch_size=16,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(300,300))

valid_generator=datagen.flow_from_dataframe(
    dataframe=labels,
    directory="Train_all/Train",
    x_col="ID",
    y_col="CLASS",
    subset="validation",
    batch_size=16,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(300,300))

num_classes = 8

Found 16149 validated image filenames belonging to 8 classes.
Found 2849 validated image filenames belonging to 8 classes.


In [5]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout,BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam

weights_2 = [ 0.7005531, 0.24592265, 0.95261733, 3.64804147, 1.20674543, 13.19375, 12.56547619, 5.04219745]
w2 = [round(w,4) for w in weights_2]
weights_2 = dict(enumerate(w2))
print(weights_2)

{0: 0.7006, 1: 0.2459, 2: 0.9526, 3: 3.648, 4: 1.2067, 5: 13.1937, 6: 12.5655, 7: 5.0422}


Model selection and training
---

In [7]:
from tensorflow.keras.applications import InceptionV3, Xception, ResNet50V2, EfficientNetV2M, EfficientNetB5

base_model = EfficientNetV2M(include_top=False, weights='imagenet', input_shape=(300, 300, 3), include_preprocessing=True)

model_t = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(num_classes, activation='softmax')
])
model_t.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-m (Function  (None, 10, 10, 1280)      53150388  
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 8)                 10248     
                                                                 
Total params: 53160636 (202.79 MB)
Trainable params: 52868604 (201.68 MB)
Non-trainable params: 292032 (1.11 MB)
_________________________________________________________________


In [8]:
lr_schedule =tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.00005,
    decay_steps=10000,
    decay_rate=0.9)

optimizer = tf.keras.optimizers.Adam(learning_rate=0.00005)

loss = "categorical_crossentropy"
metrics=[ "accuracy"]
model_t.compile(optimizer = optimizer , loss = loss, metrics = metrics)

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
earlystop = EarlyStopping(monitor="val_accuracy",patience=10, restore_best_weights=True)

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model_t.fit(train_generator,
            steps_per_epoch=STEP_SIZE_TRAIN,
            callbacks = [earlystop],
            class_weight = weights_2,
            validation_data=valid_generator,
            validation_steps=STEP_SIZE_VALID,
            epochs=25)

Epoch 1/25


2024-05-05 14:42:34.773941: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/efficientnetv2-m/block1b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-05-05 14:42:42.243234: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
2024-05-05 14:42:42.776107: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-05 14:42:43.333795: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-05-05 14:42:44.707263: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f543832ce80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-05-05 14:42:44.707312: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0):

1009/1009 [==============================] - 590s 479ms/step - loss: 1.4671 - accuracy: 0.4956 - val_loss: 1.2646 - val_accuracy: 0.5456
Epoch 2/25
1009/1009 [==============================] - 466s 462ms/step - loss: 1.0261 - accuracy: 0.6122 - val_loss: 1.3060 - val_accuracy: 0.5432
Epoch 3/25
1009/1009 [==============================] - 352s 349ms/step - loss: 0.8398 - accuracy: 0.6562 - val_loss: 1.0747 - val_accuracy: 0.6341
Epoch 4/25
1009/1009 [==============================] - 340s 336ms/step - loss: 0.7277 - accuracy: 0.6877 - val_loss: 1.0905 - val_accuracy: 0.6180
Epoch 5/25
1009/1009 [==============================] - 310s 307ms/step - loss: 0.6251 - accuracy: 0.7183 - val_loss: 1.1402 - val_accuracy: 0.5945
Epoch 6/25
1009/1009 [==============================] - 327s 324ms/step - loss: 0.5465 - accuracy: 0.7433 - val_loss: 0.9942 - val_accuracy: 0.6552
Epoch 7/25
1009/1009 [==============================] - 304s 301ms/step - loss: 0.4628 - accuracy: 0.7718 - val_loss: 1.084

In [ ]:
model_t.evaluate(valid_generator)

Predictions
---

In [11]:
testdf=pd.read_csv("submissions/sample_submission.csv",dtype=str)
testdf["ID"]=testdf["ID"].apply(append_ext)
testdf

,ID,CLASS
0,ISIC_0070035.jpg,2
1,ISIC_0030667.jpg,2
2,ISIC_0033772.jpg,8
3,ISIC_0000182.jpg,2
4,ISIC_0030206.jpg,2
...,...,...
6328,ISIC_0028453.jpg,6
6329,ISIC_0024846.jpg,2
6330,ISIC_0055816.jpg,2
6331,ISIC_0034166.jpg,2


In [12]:
test_datagen=ImageDataGenerator()
test_generator=test_datagen.flow_from_dataframe(
    dataframe=testdf,
    directory="Test",
    x_col="ID",
    y_col=None,
    batch_size=3, #must be a divisor of 6333
    seed=42,
    shuffle=False,
    class_mode=None,
    target_size=(300,300))

Found 6333 validated image filenames.


In [13]:
test_generator.reset()
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
pred=model_t.predict(test_generator,
                     steps=STEP_SIZE_TEST,
                     verbose=1)

 184/2111 [=>............................] - ETA: 45s

2111/2111 [==============================] - 51s 22ms/step


In [14]:
predicted_class_indices=np.argmax(pred,axis=1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [15]:
filenames=test_generator.filenames
filenames = [fil.split('.')[0] for fil in filenames]
results=pd.DataFrame({"ID":filenames,
                      "CLASS":predictions})
results

,ID,CLASS
0,ISIC_0070035,8
1,ISIC_0030667,2
2,ISIC_0033772,2
3,ISIC_0000182,2
4,ISIC_0030206,2
...,...,...
6328,ISIC_0028453,7
6329,ISIC_0024846,1
6330,ISIC_0055816,1
6331,ISIC_0034166,2


In [16]:
results.to_csv("submission_augm_3.csv",index=False)

Postprocessing - Majority voting
---

In [ ]:
def calculate_mode(x):
    return Counter(x).most_common(1)[0][0]

folder_path = 'submissions'

csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]
dfs = [pd.read_csv(file) for file in csv_files]
merged_df = pd.concat(dfs).groupby('ID')['CLASS'].apply(calculate_mode).reset_index()

merged_df.to_csv('merged_file.csv', index=False)